In [ ]:
import pandas as pd

from aust_covid.inputs import get_ifrs, load_pop_data
from emutools.tex import StandardTexDoc
from aust_covid.inputs import get_base_vacc_data
from aust_covid.vaccination import add_derived_data_to_vacc, get_model_vacc_vals_from_data
from aust_covid.model import build_model
from aust_covid.plotting import plot_immune_props
from inputs.constants import PROJECT_PATH, SUPPLEMENT_PATH
from emutools.parameters import load_param_info

In [ ]:
vacc_df = get_base_vacc_data()
ext_vacc_df, lagged_ext_vacc_df = add_derived_data_to_vacc(vacc_df)

In [ ]:
app_doc = StandardTexDoc(SUPPLEMENT_PATH, 'supplement', "Australia's 2023 Omicron Waves Supplement", 'austcovid')
param_info = load_param_info()
param_info['value'].update(get_ifrs(app_doc))
parameters = param_info['value'].to_dict()
epi_model = build_model(app_doc, vacc_sens=True)

In [ ]:
epi_model.run(parameters=parameters)

In [ ]:
init_pop = epi_model.get_initial_population(parameters=parameters)

In [ ]:
model_pops = load_pop_data(app_doc)
states = epi_model.stratifications['states'].strata
for age in epi_model.stratifications['agegroup'].strata:
    for state in states:
        model_pops.loc[int(age), f'check_{state}'] = init_pop[init_pop.index.str.contains(f'states_{state}') & init_pop.index.str.contains(f'agegroup_{age}X')].sum()

imm_check = {}
for imm in epi_model.stratifications['immunity'].strata:
    imm_check[imm] = init_pop[init_pop.index.str.contains(f'immunity_{imm}')].sum()

print([v / sum(imm_check.values()) for v in imm_check.values()])

In [ ]:
model_pops.plot()

In [ ]:
plot_immune_props(epi_model, ext_vacc_df, lagged_ext_vacc_df)